In [89]:
import os, json, numpy as np, pandas as pd
import seaborn as sns, matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from tweepy.api import API
from tweepy.auth import OAuthHandler
from tweepy.cursor import Cursor
from tweepy.streaming import Stream, StreamListener
from pprint import PrettyPrinter, pformat, pprint
from TweepyCredentials import ACCESS_TOKEN, ACCESS_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_KEY_SECRET

In [22]:
sns.set_style('whitegrid')
nltk.download('punkt')
nltk.download('stopwords')
%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
class Authenticator():
    '''A class that authenticates out API call.'''
    
    def authenticate_tweets(self):
        auth = OAuthHandler(CONSUMER_KEY, CONSUMER_KEY_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        return auth

In [10]:
class MyListener(StreamListener):
    '''A custom listener class inheriting from tweepy StreamListener class methods that print/write streamed tweets.'''
    
    def __init__(self, tweet_file):
        self.tweet_file = tweet_file
        self.auth = Authenticator().authenticate_tweets()
    
    def on_data(self, status):
        try:
            # main goal is to append tweets to a list or write the tweets to a file
            tweet = json.loads(status)
            with open(self.tweet_file, 'a') as tf:
                tf.write(tweet['text'])
        except BaseException as e:
            print('Error: ', str(e))

    def on_error(self, error):
        print(error)
        if error == 420:
            return False
        

In [11]:
class TwitterClient():
    '''A class to get various data from a specified user'''
    
    def __init__(self):
        self.auth = Authenticator().authenticate_tweets()
        self.client = API(self.auth)
        
    def get_user_tweets(self, num_tweets, id_):
        tweets = []
        for tweet in Cursor(self.client.user_timeline, id_).items(num_tweets):
            tweets.append(tweet._json)
            
        delete = []
        for tweet in tweets:
            for key, value in tweet.items():
                if value in [None, False]:
                    delete.append(key)    

        for key in delete:
            if key in tweet:
                del tweet[key]
            else:
                pass    
            
        return tweets

In [86]:
if __name__ == "__main__":
        
    from nltk.corpus import stopwords
    import nltk
    
    stopwords = set(stopwords.words('english'))
    
    client = TwitterClient()
    
    tweets = client.get_user_tweets(100, 'realDonaldTrump')
    tweet_text = [tweet['text'] for tweet in tweets]
#     retweet_count = [tweet['retweet_count'] for tweet in tweets]
#     created_at = [tweet['created_at'] for tweet in tweets]
        
    df = pd.DataFrame(data=tweet_text, columns=['Tweet'])
#     df['Created At'] = created_at
#     df['Retweets'] = retweet_count

#     fig = plt.figure(figsize=(15,4))
#     axes = fig.add_axes([0,0,1,1])
#     axes.plot(df['Created At'][:100], df['Retweets'][:100], label = 'Retweet trend with time')


# let's tokenize every tweet that we have got through out TwitterClient

    tokenized_tweets = []
    
    for tweet in tweet_text:
        tokenized_tweets.append(nltk.word_tokenize(tweet))
        
#     print(tokenized_tweets[0])

# let's remove noise from each tweet token

    cleaned_tokens = []
    
    for tweet_tokens in tokenized_tweets:
        cleaned_tokens.append(remove_noise(tweet_tokens, stopwords))
        
#     print(len(cleaned_tokens[0]))

# let's run our trained classifier

    sentiments = []
    
    for tweet_tokens in cleaned_tokens:
        # below code classifies a single tweet based on its tokens
        sentiments.append(classifier.classify(dict([token, True] for token in tweet_tokens)))
        
        
    print('The sentiment of {} tweets: \n'.format(len(sentiments)))
    
    df['Sentiment'] = [sentiment for sentiment in sentiments]
    
    print(df[['Tweet', 'Sentiment']])
    


The sentiment of 100 tweets: 

                                                Tweet Sentiment
0   RT @parscale: Joe Biden is a train wreck on il...  Negative
1   RT @TimMurtaugh: Joe Biden would set an extrem...  Positive
2   RT @kayleighmcenany: Sleepy Joe is SO CONFUSED...  Negative
3                             https://t.co/HGMNIEImFT  Positive
4   I must say, that was a VERY boring debate. Bid...  Negative
..                                                ...       ...
95  Many Republican Senators want me to Veto the F...  Positive
96  RT @realDonaldTrump: “Nancy Pelosi all of a su...  Negative
97  RT @TwitterMoments: The NBA has suspended the ...  Negative
98  RT @CDCemergency: Develop a plan for home-base...  Positive
99  RT @CDCgov: Everyone can help prevent the spre...  Positive

[100 rows x 2 columns]
